# Multimodal search with CLIP

In [ ]:
import clip
import pymongo
from pinnacledb.misc.pinnacle import pinnacle
from pinnacledb.models.torch.wrapper import TorchModel
from pinnacledb.datalayer.mongodb.query import Collection
from pinnacledb.core.documents import Document as D
from pinnacledb.encoders.pillow.image import pil_image as i

pymongo.MongoClient().drop_database('documents')
pymongo.MongoClient().drop_database('_filesystem:documents')

db = pymongo.MongoClient().documents
db = pinnacle(db)

collection = Collection(name='tiny-imagenet')

In [ ]:
from pinnacledb.core.documents import Document as D
from pinnacledb.encoders.pillow.image import pil_image as i
from datasets import load_dataset
import random

dataset = load_dataset("zh-plus/tiny-imagenet")['valid']
dataset = [D({'image': i(r['image'])}) for r in dataset]
dataset = random.sample(dataset, 1000)

In [ ]:
db.execute(collection.insert_many(dataset, encoders=(i,)))

In [ ]:
x = db.execute(collection.find_one())['image'].x

In [ ]:
model, preprocess = clip.load("ViT-B/32", device='cpu')

In [ ]:
from pinnacledb.encoders.torch.tensor import tensor
import torch

t = tensor(torch.float, shape=(512,))

In [ ]:
text_model = TorchModel(
    identifier='clip_text',
    object=model,
    preprocess=lambda x: clip.tokenize(x)[0],
    forward_method='encode_text',
    encoder=t
)

In [ ]:
text_model.predict('this is a test', one=True)

In [ ]:
visual_model = TorchModel(
    identifier='clip_image',
    preprocess=preprocess,
    object=model.visual,
    encoder=t,
)

In [ ]:
visual_model.predict(x, one=True)

In [ ]:
from pinnacledb.core.vector_index import VectorIndex
from pinnacledb.core.watcher import Watcher

db.add(
    VectorIndex(
        'my-index',
        indexing_watcher=Watcher(
            model=visual_model,
            key='image',
            select=collection.find(),
        ),
        compatible_watcher=Watcher(
            model=text_model,
            key='text',
            active=False,
        )
    )
)

In [ ]:
out = db.execute(
    collection.like(D({'text': 'mushroom'}), vector_index='my-index', n=3).find({})
)

In [ ]:
from IPython.display import display
for r in out:
    display(r['image'].x)